# Đồ án Học máy thống kê (DS102)
Thành viên thực hiện:
- Thái Bình Dương - 23520356
- Phạm Bá Thuận - 22521447
- Trương Tất Quang Vinh - 22521683

Các mô hình được sử dụng:
- Hồi quy Logistic
- Hồi quy Logistic với hàm mất mát dùng SGD
- SVM
- SVM với hàm mất mát dùng SGD

In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('full_dataset.csv')

In [3]:
df.head()

,Title,Source,Topic,Context/Summary,Nội dung,Từ ngữ,Dấu câu,Số từ,Total
0,Hải Phòng: Bắt 2 đối tượng chuyên trộm cắp xe máy,Lao Động,Pháp Luật,"Công an TP Hải Phòng cho biết, vừa bắt giữ 2 đ...",0,0,0,0,0
1,Vì sao giới trẻ Trung Quốc sẵn sàng ăn mặc “th...,Lao Động,Gia Đình - Hôn Nhân,Hình ảnh những cô gái mặc váy siêu ngắn quảng ...,0,1,1,0,1
2,Trương Ngọc Ánh nghiện rượu nặng,afamily.vn,Giải Trí,Trương Ngọc Ánh nhận vai diễn là 1 nhân vật ng...,1,0,0,0,1
3,Con người đã làm sạch cơ thể như thế nào khi c...,GenK,Sống,TPO - Những ghi chép sớm nhất về các chất giốn...,0,0,1,0,1
4,Vĩnh Phúc phấn đấu trở thành điển hình cho hệ ...,Nhân Dân,Đồng Bằng Sông Hồng,"Sáng 20/12, tỉnh Vĩnh Phúc phối hợp Bộ Khoa họ...",0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3908 entries, 0 to 3907
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            3908 non-null   object
 1   Source           3908 non-null   object
 2   Topic            3908 non-null   object
 3   Context/Summary  3908 non-null   object
 4   Nội dung         3908 non-null   int64 
 5   Từ ngữ           3908 non-null   int64 
 6   Dấu câu          3908 non-null   int64 
 7   Số từ            3908 non-null   int64 
 8   Total            3908 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 274.9+ KB


In [5]:
df['Total'].value_counts()

Total
0    1954
1    1954
Name: count, dtype: int64

## Phương pháp 1: Dùng mô hình BERT biến đổi về token rồi phân loại

Sử dụng tokenizer của mô hình BERT để chuyển thành các dạng vector đặc trưng

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Hàm chuyển đổi văn bản thành vector đặc trưng
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    # Sử dụng embedding của token [CLS]
    return outputs.last_hidden_state[:, 0, :].detach().numpy().flatten()

df['title_embedding'] = df['Title'].apply(lambda x: encode_text(x))
df['context_embedding'] = df['Context/Summary'].apply(lambda x: encode_text(x))

In [7]:
df.head()

,Title,Source,Topic,Context/Summary,Nội dung,Từ ngữ,Dấu câu,Số từ,Total,title_embedding,context_embedding
0,Hải Phòng: Bắt 2 đối tượng chuyên trộm cắp xe máy,Lao Động,Pháp Luật,"Công an TP Hải Phòng cho biết, vừa bắt giữ 2 đ...",0,0,0,0,0,"[-0.71299726, 0.03100263, 0.20968026, -0.26313...","[-0.8575342, -0.08392504, 0.19292606, -0.35398..."
1,Vì sao giới trẻ Trung Quốc sẵn sàng ăn mặc “th...,Lao Động,Gia Đình - Hôn Nhân,Hình ảnh những cô gái mặc váy siêu ngắn quảng ...,0,1,1,0,1,"[-0.80902183, -0.011687051, -0.0059735402, -0....","[-1.176662, -0.003726624, 0.1798513, -0.432526..."
2,Trương Ngọc Ánh nghiện rượu nặng,afamily.vn,Giải Trí,Trương Ngọc Ánh nhận vai diễn là 1 nhân vật ng...,1,0,0,0,1,"[-0.77212703, 0.018759597, -0.009384301, -0.38...","[-0.6397566, -0.080910966, 0.13438088, -0.4164..."
3,Con người đã làm sạch cơ thể như thế nào khi c...,GenK,Sống,TPO - Những ghi chép sớm nhất về các chất giốn...,0,0,1,0,1,"[-0.60842055, -0.13040267, -0.0401419, -0.4098...","[-1.0008287, 0.03606192, 0.39984372, -0.274794..."
4,Vĩnh Phúc phấn đấu trở thành điển hình cho hệ ...,Nhân Dân,Đồng Bằng Sông Hồng,"Sáng 20/12, tỉnh Vĩnh Phúc phối hợp Bộ Khoa họ...",0,0,0,0,0,"[-1.0218565, 0.11979359, -0.09953272, -0.20461...","[-0.6136223, -0.6126117, 0.2358214, -0.3825055..."


In [12]:
X = np.hstack([
        np.array(df['title_embedding'].to_list()),  # Chuyển thành numpy array
        np.array(df['context_embedding'].to_list())
    ])
y = df[['Total']]

# Chia dữ liệu
# Lấy 2500 dòng đầu cho tập train
X_train = X[:2500]
y_train = y[:2500]

# Lấy phần còn lại và chia đều cho validation và test
X_remaining = X[2500:]
y_remaining = y[2500:]
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

In [4]:
# Hàm chạy mô hình và ghi kết quả
def evaluate_model(model, X, y, set_name):
        y_pred = model.predict(X)
        print(f"\nKết quả trên tập {set_name}:")
        print(classification_report(y, y_pred, digits=3))

SVM

In [17]:
from sklearn.model_selection import GridSearchCV

svm_bert = SVC(C=5, kernel='rbf', random_state=42)
svm_bert.fit(X_train, y_train)

evaluate_model(svm_bert, X_train, y_train, "huấn luyện")
evaluate_model(svm_bert, X_val, y_val, "validation")
evaluate_model(svm_bert, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.869     0.897     0.883      1262
           1      0.891     0.862     0.876      1238

    accuracy                          0.880      2500
   macro avg      0.880     0.879     0.880      2500
weighted avg      0.880     0.880     0.880      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.766     0.871     0.815       342
           1      0.860     0.749     0.801       362

    accuracy                          0.808       704
   macro avg      0.813     0.810     0.808       704
weighted avg      0.815     0.808     0.808       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.763     0.820     0.791       350
           1      0.808     0.749     0.777       354

    accuracy                          0.784       704
   macro avg      0.786     0.784     0.

SVM-SGD

In [28]:
# Huấn luyện mô hình SVM bằng SGDClassifier
svm_sgd_model = SGDClassifier(loss='hinge', learning_rate='optimal', alpha=0.025)
svm_sgd_model.fit(X_train, y_train)

# Dự đoán và tính toán độ chính xác
evaluate_model(svm_sgd_model, X_train, y_train, "huấn luyện")
evaluate_model(svm_sgd_model, X_val, y_val, "validation")
evaluate_model(svm_sgd_model, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.845     0.854     0.850      1262
           1      0.850     0.841     0.845      1238

    accuracy                          0.848      2500
   macro avg      0.848     0.848     0.848      2500
weighted avg      0.848     0.848     0.848      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.764     0.842     0.801       342
           1      0.835     0.754     0.792       362

    accuracy                          0.797       704
   macro avg      0.799     0.798     0.797       704
weighted avg      0.800     0.797     0.797       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.780     0.800     0.790       350
           1      0.797     0.777     0.787       354

    accuracy                          0.788       704
   macro avg      0.789     0.788     0.

Logistic

In [19]:
lr_model = LogisticRegression(solver='liblinear',C=0.1, random_state=42)
lr_model.fit(X_train, y_train)

evaluate_model(lr_model, X_train, y_train, "huấn luyện")
evaluate_model(lr_model, X_val, y_val, "validation")
evaluate_model(lr_model, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.861     0.896     0.878      1262
           1      0.890     0.852     0.870      1238

    accuracy                          0.874      2500
   macro avg      0.875     0.874     0.874      2500
weighted avg      0.875     0.874     0.874      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.749     0.854     0.798       342
           1      0.841     0.729     0.781       362

    accuracy                          0.790       704
   macro avg      0.795     0.792     0.789       704
weighted avg      0.796     0.790     0.789       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.768     0.803     0.785       350
           1      0.796     0.760     0.777       354

    accuracy                          0.781       704
   macro avg      0.782     0.781     0.

Logistic SGD

In [46]:
# Huấn luyện mô hình Logistic Regression bằng SGDClassifier
lr_sgd_model = SGDClassifier(loss='log_loss', learning_rate='optimal', alpha=0.01, random_state=42)
lr_sgd_model.fit(X_train, y_train)

evaluate_model(lr_sgd_model, X_train, y_train, "huấn luyện")
evaluate_model(lr_sgd_model, X_val, y_val, "validation")
evaluate_model(lr_sgd_model, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.806     0.903     0.852      1262
           1      0.888     0.778     0.829      1238

    accuracy                          0.841      2500
   macro avg      0.847     0.841     0.840      2500
weighted avg      0.846     0.841     0.841      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.729     0.895     0.803       342
           1      0.873     0.685     0.768       362

    accuracy                          0.787       704
   macro avg      0.801     0.790     0.785       704
weighted avg      0.803     0.787     0.785       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.739     0.851     0.792       350
           1      0.827     0.703     0.760       354

    accuracy                          0.777       704
   macro avg      0.783     0.777     0.

## Phương pháp 2: Sử dụng TF-IDF

In [7]:
def get_strong_patterns():
    """Định nghĩa các mẫu từ và cụm từ mạnh"""
    return {
        'strong_verbs': [
            'sốc', 'choáng', 'ngỡ ngàng', 'bất ngờ', 'kinh hoàng', 'khủng khiếp',
            'phẫn nộ', 'căng thẳng', 'tức giận', 'náo loạn', 'hoảng loạn',
            'phát hiện', 'tiết lộ', 'tố cáo', 'vạch trần', 'lật tẩy',
            'đột phá', 'đột kích', 'đột nhập', 'tấn công', 'khởi tố',
            'bắt quả tang', 'bắt giữ', 'triệt phá', 'xóa sổ', 'san phẳng'
        ],
        'strong_adjectives': [
            'khủng', 'khổng lồ', 'siêu', 'cực', 'đỉnh', 'xịn',
            'kinh dị', 'kinh khủng', 'dễ sợ', 'đáng sợ', 'rùng rợn',
            'gây sốc', 'sốc nặng', 'không tưởng', 'khó tin',
            'đắt đỏ', 'rẻ bèo', 'bất thường', 'lạ lùng', 'kỳ lạ'
        ],
        'clickbait_phrases': [
            'và cái kết', 'bất ngờ với', 'tá hỏa', 'ai cũng phải',
            'không thể tin', 'khiến dân tình', 'gây bão', 'xuất hiện bất ngờ',
            'cư dân mạng', 'dân mạng', 'cộng đồng mạng', 'netizen'
        ]
    }

# Chuyển đổi text thành features
def convert_text_to_features(title_text, content_text):
    """Chuyển đổi text thành features số, chỉ áp dụng cho tiêu đề"""
    title_text = str(title_text)
    content_text = str(content_text)  # Giữ lại để phân tích tính chính xác
    
    patterns = get_strong_patterns()

    features = {
        # Đặc trưng về độ dài tiêu đề
        'title_length': len(title_text),
        'title_word_count': len(title_text.split()),
        
        # Đặc trưng về dấu câu trong tiêu đề
        'title_exclamation': title_text.count('!'),
        'title_question': title_text.count('?'),
        'title_dots': title_text.count('...'),
        'title_colon': title_text.count(':'),
        'title_single_quote': title_text.count("'"),
        'title_double_quote': title_text.count('"'),

        # Đặc trưng về từ mạnh trong tiêu đề
        'title_strong_verbs': sum(1 for word in patterns['strong_verbs'] if word in title_text.lower()),
        'title_strong_adjectives': sum(1 for word in patterns['strong_adjectives'] if word in title_text.lower()),
        'title_clickbait_phrases': sum(1 for phrase in patterns['clickbait_phrases'] if phrase in title_text.lower()),
        
        # Đặc trưng về số trong tiêu đề
        'title_numbers': len([n for n in title_text.split() if n.isdigit()]),
        
        # Đặc trưng về chữ hoa trong tiêu đề
        'title_uppercase_ratio': sum(1 for c in title_text if c.isupper()) / len(title_text) if len(title_text) > 0 else 0,
    }
    return features

In [126]:
# Tạo TF-IDF cho tiêu đề
title_tfidf = TfidfVectorizer(max_features=2000)
title_features = title_tfidf.fit_transform(df['Title'].astype(str))

# Tạo TF-IDF cho nội dung
content_tfidf = TfidfVectorizer(max_features=2000)
content_features = content_tfidf.fit_transform(df['Context/Summary'].astype(str))

# Trích xuất đặc trưng bổ sung
additional_features = pd.DataFrame([
    convert_text_to_features(title, content) 
    for title, content in zip(df['Title'], df['Context/Summary'])
])

X = np.hstack([
        title_features.toarray(),
        content_features.toarray(),
        additional_features
    ])
y = df[['Total']]

# Chia dữ liệu
# Lấy 2500 dòng đầu cho tập train
X_train = X[:2500]
y_train = y[:2500]

# Lấy phần còn lại và chia đều cho validation và test
X_remaining = X[2500:]
y_remaining = y[2500:]
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

SVM

In [127]:
best_svm = SVC(C=1000, kernel='rbf', random_state=42)
best_svm.fit(X_train, y_train)

# Đánh giá mô hình
evaluate_model(best_svm, X_train, y_train, "huấn luyện")
evaluate_model(best_svm, X_val, y_val, "validation")
evaluate_model(best_svm, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.902     0.951     0.926      1262
           1      0.947     0.894     0.920      1238

    accuracy                          0.923      2500
   macro avg      0.924     0.923     0.923      2500
weighted avg      0.924     0.923     0.923      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.757     0.886     0.817       342
           1      0.872     0.732     0.796       362

    accuracy                          0.807       704
   macro avg      0.815     0.809     0.806       704
weighted avg      0.816     0.807     0.806       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.759     0.857     0.805       350
           1      0.838     0.732     0.781       354

    accuracy                          0.794       704
   macro avg      0.799     0.794     0.

SVM-SGD

In [128]:
best_sgd_svm = SGDClassifier(loss='hinge', alpha=0.01, penalty='l1', random_state=42)
best_sgd_svm.fit(X_train, y_train)

# Đánh giá mô hình SGD-SVM
print("\nKết quả SGD-SVM:")
evaluate_model(best_sgd_svm, X_train, y_train, "huấn luyện")
evaluate_model(best_sgd_svm, X_val, y_val, "validation")
evaluate_model(best_sgd_svm, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả SGD-SVM:

Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.780     0.793     0.787      1262
           1      0.786     0.772     0.779      1238

    accuracy                          0.783      2500
   macro avg      0.783     0.783     0.783      2500
weighted avg      0.783     0.783     0.783      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.759     0.784     0.771       342
           1      0.789     0.765     0.777       362

    accuracy                          0.774       704
   macro avg      0.774     0.774     0.774       704
weighted avg      0.775     0.774     0.774       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.741     0.777     0.759       350
           1      0.769     0.732     0.750       354

    accuracy                          0.754       704
   macro avg      0.75

Logistic

In [129]:
best_logistic = LogisticRegression(C=1, penalty='l1', solver='liblinear', random_state=42)
best_logistic.fit(X_train, y_train)

# Đánh giá mô hình
evaluate_model(best_logistic, X_train, y_train, "huấn luyện")
evaluate_model(best_logistic, X_val, y_val, "validation")
evaluate_model(best_logistic, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.815     0.891     0.851      1262
           1      0.877     0.794     0.833      1238

    accuracy                          0.843      2500
   macro avg      0.846     0.842     0.842      2500
weighted avg      0.846     0.843     0.842      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.771     0.886     0.824       342
           1      0.875     0.751     0.808       362

    accuracy                          0.817       704
   macro avg      0.823     0.819     0.816       704
weighted avg      0.824     0.817     0.816       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.755     0.846     0.798       350
           1      0.827     0.729     0.775       354

    accuracy                          0.787       704
   macro avg      0.791     0.787     0.

Logistic-SGD

In [ ]:
best_sgd_log = SGDClassifier(loss='log_loss', alpha=0.01)
best_sgd_log.fit(X_train, y_train)

# Đánh giá mô hình SGD-Logistic
print("\nKết quả SGD-Logistic:")
evaluate_model(best_sgd_log, X_train, y_train, "huấn luyện")
evaluate_model(best_sgd_log, X_val, y_val, "validation")
evaluate_model(best_sgd_log, X_test, y_test, "kiểm thử")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Kết quả SGD-Logistic:

Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.735     0.834     0.781      1262
           1      0.803     0.693     0.744      1238

    accuracy                          0.764      2500
   macro avg      0.769     0.763     0.763      2500
weighted avg      0.769     0.764     0.763      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.708     0.810     0.756       342
           1      0.792     0.685     0.735       362

    accuracy                          0.746       704
   macro avg      0.750     0.748     0.745       704
weighted avg      0.752     0.746     0.745       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.706     0.809     0.754       350
           1      0.779     0.667     0.718       354

    accuracy                          0.737       704
   macro avg     

## Phương pháp 3: Dùng hệ số LIX và RIX

In [5]:
# Tính toán LIX và RIX
def calculate_lix(text):
    words = text.split()
    num_words = len(words)
    num_sentences = len(re.findall(r'[.!?]', text))
    num_long_words = len([word for word in words if len(word) > 6])
    
    if num_sentences == 0:
        return 0
    
    lix = num_words / num_sentences + (num_long_words * 100) / num_words
    return lix

def calculate_rix(text):
    words = text.split()
    num_sentences = len(re.findall(r'[.!?]', text))
    num_long_words = len([word for word in words if len(word) > 6])
    
    if num_sentences == 0:
        return 0
    
    rix = num_long_words / num_sentences
    return rix

In [8]:
df['RIX'] = df['Title'].apply(calculate_lix)
df['LIX'] = df['Context/Summary'].apply(calculate_rix)
title_features = df['RIX'].values.reshape(-1, 1)
content_features = df['LIX'].values.reshape(-1, 1)

# Trích xuất đặc trưng bổ sung
additional_features = pd.DataFrame([
    convert_text_to_features(title, content) 
    for title, content in zip(df['Title'], df['Context/Summary'])
])

# Kết hợp tất cả features
X = np.hstack([
    title_features,
    content_features,
    additional_features
])
y = df['Total']

# Chia dữ liệu
# Lấy 2500 dòng đầu cho tập train
X_train = X[:2500]
y_train = y[:2500]

# Lấy phần còn lại và chia đều cho validation và test
X_remaining = X[2500:]
y_remaining = y[2500:]
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

SVM

In [9]:
svm_model = SVC(C=750, kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)

evaluate_model(svm_model, X_train, y_train, "huấn luyện")
evaluate_model(svm_model, X_val, y_val, "validation")
evaluate_model(svm_model, X_test, y_test, "kiểm thử")


Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.793     0.791     0.792      1262
           1      0.787     0.789     0.788      1238

    accuracy                          0.790      2500
   macro avg      0.790     0.790     0.790      2500
weighted avg      0.790     0.790     0.790      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.768     0.792     0.780       342
           1      0.798     0.773     0.785       362

    accuracy                          0.783       704
   macro avg      0.783     0.783     0.783       704
weighted avg      0.783     0.783     0.783       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.761     0.771     0.766       350
           1      0.771     0.760     0.765       354

    accuracy                          0.766       704
   macro avg      0.766     0.766     0.

SVM-SGD

In [229]:
sgd_svm = SGDClassifier(loss='hinge', alpha=0.1)
sgd_svm.fit(X_train, y_train)

# Đánh giá mô hình SGD-Logistic
print("\nKết quả SGD-Logistic:")
evaluate_model(sgd_svm, X_train, y_train, "huấn luyện")
evaluate_model(sgd_svm, X_val, y_val, "validation")
evaluate_model(sgd_svm, X_test, y_test, "kiểm thử")


Kết quả SGD-Logistic:

Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.739     0.807     0.772      1262
           1      0.783     0.710     0.745      1238

    accuracy                          0.759      2500
   macro avg      0.761     0.759     0.758      2500
weighted avg      0.761     0.759     0.759      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.717     0.792     0.753       342
           1      0.782     0.704     0.741       362

    accuracy                          0.747       704
   macro avg      0.750     0.748     0.747       704
weighted avg      0.750     0.747     0.747       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.709     0.786     0.745       350
           1      0.763     0.681     0.719       354

    accuracy                          0.733       704
   macro avg     

Logistic

In [14]:
logistic_model = LogisticRegression(C=0.25, penalty='l1', solver='liblinear' ,random_state=42)
logistic_model.fit(X_train, y_train)

evaluate_model(logistic_model, X_train, y_train, "huấn luyện")
evaluate_model(logistic_model, X_val, y_val, "validation")
evaluate_model(logistic_model, X_test, y_test, "kiểm thử")


Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.786     0.820     0.803      1262
           1      0.808     0.773     0.790      1238

    accuracy                          0.797      2500
   macro avg      0.797     0.797     0.797      2500
weighted avg      0.797     0.797     0.797      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.759     0.822     0.789       342
           1      0.817     0.754     0.784       362

    accuracy                          0.787       704
   macro avg      0.788     0.788     0.787       704
weighted avg      0.789     0.787     0.787       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.751     0.783     0.766       350
           1      0.776     0.743     0.759       354

    accuracy                          0.763       704
   macro avg      0.763     0.763     0.

Logistic-SGD

In [223]:
sgd_log = SGDClassifier(loss='log_loss', alpha=0.001)
sgd_log.fit(X_train, y_train)

# Đánh giá mô hình SGD-Logistic
print("\nKết quả SGD-Logistic:")
evaluate_model(sgd_log, X_train, y_train, "huấn luyện")
evaluate_model(sgd_log, X_val, y_val, "validation")
evaluate_model(sgd_log, X_test, y_test, "kiểm thử")


Kết quả SGD-Logistic:

Kết quả trên tập huấn luyện:
              precision    recall  f1-score   support

           0      0.801     0.748     0.774      1262
           1      0.759     0.811     0.784      1238

    accuracy                          0.779      2500
   macro avg      0.780     0.780     0.779      2500
weighted avg      0.781     0.779     0.779      2500


Kết quả trên tập validation:
              precision    recall  f1-score   support

           0      0.771     0.757     0.764       342
           1      0.774     0.787     0.781       362

    accuracy                          0.773       704
   macro avg      0.773     0.772     0.772       704
weighted avg      0.773     0.773     0.773       704


Kết quả trên tập kiểm thử:
              precision    recall  f1-score   support

           0      0.775     0.729     0.751       350
           1      0.747     0.791     0.768       354

    accuracy                          0.760       704
   macro avg     